# **BÁO CÁO BÀI THỰC HÀNH**
## Sequence Representation trong RNN (Integer Encoding, One-hot Encoding, Padding, Truncation, Embedding)

### **1. Mục tiêu bài thực hành**
- Hiểu cách dữ liệu dạng chuỗi (text) được biến đổi thành dạng số để đưa vào mô hình học sâu (Deep Learning).
- Thực hành các kỹ thuật biểu diễn chuỗi phổ biến trong RNN:
  - Integer Encoding
  - One-hot Encoding
  - Padding
  - Truncation
  - Word Embedding
- Thấy được sự khác biệt giữa các phương pháp mã hoá và lý do tại sao Embedding được sử dụng trong mô hình NLP.

---

## Nội dung chính

1. Thiết lập dữ liệu mẫu và import thư viện
2. Tạo vocabulary (word2idx / idx2word)
3. Integer encoding
4. One-hot encoding (ví dụ)
5. Padding & Truncation
6. Tạo Embedding và kiểm tra
7. Forward qua RNN (chỉ forward)
8. Một bước training demo (next-token prediction)

---


## 1) Thiết lập & dữ liệu mẫu

Giải thích: Khởi tạo dữ liệu đầu vào (một vài câu), import thư viện cần thiết.

In [1]:
# Cell 1: Import thư viện và dữ liệu mẫu
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from typing import List

# Dữ liệu mẫu (nhỏ, dễ theo dõi)
sentences = ["I Can Do It", "I Try So Hard"]

print("Sentences:", sentences)

Sentences: ['I Can Do It', 'I Try So Hard']


## 2) Tạo vocabulary (Integer Encoding)

Giải thích: Tạo tập từ vựng (unique tokens), ánh xạ từ→index và index→từ. Reserve index 0 cho token `<PAD>` để tiện padding.

In [2]:
# Cell 2: Tạo vocab
tokens = " ".join(sentences).split()
unique_tokens = sorted(set(tokens))

# word2idx: từ -> index. index 0 dành cho <PAD>.
word2idx = {word: idx + 1 for idx, word in enumerate(unique_tokens)}
word2idx["<PAD>"] = 0

# idx2word: index -> từ (hữu ích khi debug)
idx2word = {idx: word for word, idx in word2idx.items()}

print("Unique tokens:", unique_tokens)
print("word2idx:", word2idx)
print("idx2word:", idx2word)

Unique tokens: ['Can', 'Do', 'Hard', 'I', 'It', 'So', 'Try']
word2idx: {'Can': 1, 'Do': 2, 'Hard': 3, 'I': 4, 'It': 5, 'So': 6, 'Try': 7, '<PAD>': 0}
idx2word: {1: 'Can', 2: 'Do', 3: 'Hard', 4: 'I', 5: 'It', 6: 'So', 7: 'Try', 0: '<PAD>'}


## 3) Integer encoding

Giải thích: Chuyển mỗi câu thành danh sách index theo `word2idx`.

In [3]:
# Cell 3: Integer encode sentences
integer_encoded: List[List[int]] = []
for sent in sentences:
    ints = [word2idx[word] for word in sent.split()]
    integer_encoded.append(ints)

print("Original sentences:", sentences)
print("Integer encoded:", integer_encoded)

Original sentences: ['I Can Do It', 'I Try So Hard']
Integer encoded: [[4, 1, 2, 5], [4, 7, 6, 3]]


## 4) One-hot encoding (ví dụ)

Giải thích: One-hot tạo vector nhị phân cho mỗi token. Thường ít dùng trực tiếp cho DL do tính sparse.

In [4]:
# Cell 4: One-hot encoding example
vocab_size = len(word2idx)  # bao gồm <PAD>
def one_hot_encode(index: int, vocab_size: int) -> torch.Tensor:
    vec = torch.zeros(vocab_size, dtype=torch.float32)
    vec[index] = 1.0
    return vec

print("Vocab size (including <PAD>):", vocab_size)
print("\nOne-hot vectors for the first sentence:")
for idx in integer_encoded[0]:
    print(f"Index {idx} ('{idx2word[idx]}') ->", one_hot_encode(idx, vocab_size))

Vocab size (including <PAD>): 8

One-hot vectors for the first sentence:
Index 4 ('I') -> tensor([0., 0., 0., 0., 1., 0., 0., 0.])
Index 1 ('Can') -> tensor([0., 1., 0., 0., 0., 0., 0., 0.])
Index 2 ('Do') -> tensor([0., 0., 1., 0., 0., 0., 0., 0.])
Index 5 ('It') -> tensor([0., 0., 0., 0., 0., 1., 0., 0.])


## 5) Padding sequences

Giải thích: Để batch các sequence có độ dài khác nhau, ta padding token `<PAD>` (index 0) để đưa về cùng chiều dài.

In [5]:
# Cell 5: Padding
tensor_sequences = [torch.tensor(seq, dtype=torch.long) for seq in integer_encoded]
padded_sequences = pad_sequence(tensor_sequences, batch_first=True, padding_value=word2idx["<PAD>"])

print("Tensor sequences (before padding):", tensor_sequences)
print("\nPadded sequences (batch, seq_len):\n", padded_sequences)
print("Padded shape:", padded_sequences.shape)

Tensor sequences (before padding): [tensor([4, 1, 2, 5]), tensor([4, 7, 6, 3])]

Padded sequences (batch, seq_len):
 tensor([[4, 1, 2, 5],
        [4, 7, 6, 3]])
Padded shape: torch.Size([2, 4])


## 6) Truncation (cắt ngắn)

Giải thích: Nếu muốn giới hạn chiều dài tối đa để tiết kiệm bộ nhớ hoặc phù hợp với mô hình, ta có thể cắt ngắn (truncate).

In [6]:
# Cell 6: Truncation example
max_len = 2
truncated = [seq[:max_len] for seq in integer_encoded]

tensor_trunc = [torch.tensor(seq, dtype=torch.long) for seq in truncated]
padded_trunc = pad_sequence(tensor_trunc, batch_first=True, padding_value=word2idx["<PAD>"])

print("Truncated sequences (max_len={}):".format(max_len), truncated)
print("Padded truncated sequences:", padded_trunc)

Truncated sequences (max_len=2): [[4, 1], [4, 7]]
Padded truncated sequences: tensor([[4, 1],
        [4, 7]])


## 7) Embedding layer

Giải thích: Embedding ánh xạ index thành vector dense (học được). Sử dụng `padding_idx` để embedding của PAD không bị cập nhật.

In [7]:
# Cell 7: Embedding
embedding_dim = 8
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=word2idx["<PAD>"])

embedded = embedding(padded_sequences)  # shape: (batch, seq_len, embedding_dim)

print("Embedding matrix shape (num_embeddings, embedding_dim):", embedding.weight.shape)
print("Embedded output shape (batch, seq_len, emb_dim):", embedded.shape)
print("Example embedded vector (sentence 0, token 0):", embedded[0,0])

Embedding matrix shape (num_embeddings, embedding_dim): torch.Size([8, 8])
Embedded output shape (batch, seq_len, emb_dim): torch.Size([2, 4, 8])
Example embedded vector (sentence 0, token 0): tensor([ 0.3942,  0.7725, -0.2003,  0.5454, -1.5207, -0.8954, -2.2208, -0.6201],
       grad_fn=<SelectBackward0>)


## 8) RNN forward (chỉ forward)

Giải thích: Đưa embedding vào RNN để quan sát output shapes. Ở đây dùng `batch_first=True` nên input shape là (batch, seq_len, features).

In [8]:
# Cell 8: RNN forward
hidden_size = 16
rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1, batch_first=True)

rnn_out, rnn_hidden = rnn(embedded)

print("RNN out shape (batch, seq_len, hidden_size):", rnn_out.shape)
print("RNN hidden shape (num_layers, batch, hidden_size):", rnn_hidden.shape)

RNN out shape (batch, seq_len, hidden_size): torch.Size([2, 4, 16])
RNN hidden shape (num_layers, batch, hidden_size): torch.Size([1, 2, 16])


## 9) Một bước training demo (Next-token prediction)

Giải thích: Chuẩn bị targets bằng cách shift input sang trái (target tại time t là token ở t+1). Dùng CrossEntropyLoss với `ignore_index` để bỏ qua PAD.

In [9]:
# Cell 9: One training step demo
input_indices = padded_sequences  # (batch, seq_len)

# Prepare targets by shifting left (next-token)
targets = torch.zeros_like(input_indices)
targets[:, :-1] = input_indices[:, 1:]
targets[:, -1] = word2idx["<PAD>"]

class SimpleRNNModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, padding_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=padding_idx)
        self.rnn = nn.RNN(input_size=emb_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        emb = self.embedding(x)
        out, hidden = self.rnn(emb)
        logits = self.fc(out)
        return logits

vocab_size = len(word2idx)
model = SimpleRNNModel(vocab_size=vocab_size, emb_dim=embedding_dim, hidden_dim=hidden_size, padding_idx=word2idx["<PAD>"])
criterion = nn.CrossEntropyLoss(ignore_index=word2idx["<PAD>"])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# One training step
model.train()
optimizer.zero_grad()
logits = model(input_indices)  # (batch, seq_len, vocab_size)
B, S, V = logits.shape
loss = criterion(logits.view(B*S, V), targets.view(B*S))
loss.backward()
optimizer.step()

print("One training step done. Loss:", loss.item())

One training step done. Loss: 2.2267966270446777
